In [21]:
import pandas as pd
melb_data = pd.read_csv('data\melb_data_fe.csv', sep=',')
melb_df = melb_data.copy()
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Suburb           13580 non-null  object 
 1   Rooms            13580 non-null  int64  
 2   Type             13580 non-null  object 
 3   Price            13580 non-null  float64
 4   Method           13580 non-null  object 
 5   SellerG          13580 non-null  object 
 6   Date             13580 non-null  object 
 7   Distance         13580 non-null  float64
 8   Postcode         13580 non-null  int64  
 9   Bedroom          13580 non-null  int64  
 10  Bathroom         13580 non-null  int64  
 11  Car              13580 non-null  int64  
 12  Landsize         13580 non-null  float64
 13  BuildingArea     13580 non-null  float64
 14  CouncilArea      12211 non-null  object 
 15  Lattitude        13580 non-null  float64
 16  Longtitude       13580 non-null  float64
 17  Regionname  

None

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
#display(melb_df['Date'])
#разделение на кварталы
quartal_time = melb_df['Date'].dt.quarter
#вывод количества продаж по квартально
quartal_time.value_counts()

In [22]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Suburb           13580 non-null  category
 1   Rooms            13580 non-null  int64   
 2   Type             13580 non-null  category
 3   Price            13580 non-null  float64 
 4   Method           13580 non-null  category
 5   SellerG          13580 non-null  category
 6   Date             13580 non-null  object  
 7   Distance         13580 non-null  float64 
 8   Postcode         13580 non-null  int64   
 9   Bedroom          13580 non-null  int64   
 10  Bathroom         13580 non-null  int64   
 11  Car              13580 non-null  int64   
 12  Landsize         13580 non-null  float64 
 13  BuildingArea     13580 non-null  float64 
 14  CouncilArea      12211 non-null  category
 15  Lattitude        13580 non-null  float64 
 16  Longtitude       13580 non-null  float64

None

In [ ]:
#сортировка недвижимости по цене
melb_df.sort_values(by='Price').head(10)

In [ ]:
# сортировка по дате (с самой последне до первой) параметр ascending=False
melb_df.sort_values(by='Date', ascending=False)

In [ ]:
#сортировка сначала по дистанции потом по цене
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

In [ ]:
#соотношения площадей здания и участка (AreaRatio) меньше -0.8
mask1 = melb_df['AreaRatio'] < -0.8
#Найдём информацию о таунхаусах (Type)
mask2 = melb_df['Type'] == 'townhouse'
#проданных компанией (SellerG) McGrath
mask3 = melb_df['SellerG'] == 'McGrath'
#Результат отсортируем по дате продажи (Date) в порядке возрастания. 
#Также обновим старые индексы на новые, установив параметр ignore_index на True
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

In [27]:
melb_df.sort_values(by=['AreaRatio'], ascending=False, ignore_index=True).loc[1558, ['BuildingArea']]


BuildingArea    126.0
Name: 1558, dtype: object

Задача

In [30]:
mask1 = melb_df['Rooms'] > 2
mask3 = melb_df['Type'] == 'townhouse'
melb_df[mask1 & mask3].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, ['Price']]

Price    1300000.0
Name: 18, dtype: object

Группировка

In [31]:
melb_df.groupby(by='Type').mean()

,Rooms,Price,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,Lattitude,Longtitude,Propertycount,MeanRoomsSquare,AreaRatio,AgeBuilding
Type,,,,,,,,,,,,,,,
house,3.260874,1.242665e+06,10.979479,3104.080643,3.229336,1.613822,1.772674,617.181924,152.162553,-37.803795,144.994700,7259.025505,18.996731,-0.490031,55.669700
townhouse,2.837522,9.337351e+05,9.851346,3100.777379,2.814183,1.809695,1.555655,279.606822,134.649710,-37.815782,144.996489,7094.459605,18.569847,-0.094916,26.690305
unit,1.963871,6.051275e+05,7.607391,3110.797481,1.966523,1.183295,1.128936,477.314219,102.235863,-37.823710,144.996363,8199.280080,21.068242,0.319883,39.703016


In [32]:
melb_df.groupby('Type')['Price'].mean()

Type
house        1.242665e+06
townhouse    9.337351e+05
unit         6.051275e+05
Name: Price, dtype: float64

In [33]:
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

In [ ]:
#найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price),
#сгруппированных по номеру месяца продажи (MonthSale). 
#Результат отсортируем по количеству продаж в порядке убывания
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg('describe')

In [36]:
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{Fletchers, Miles, Philip, C21, Ray, Jellis, M..."
Eastern Victoria,11,"{Barry, McGrath, Fletchers, Ray, O'Brien, Harc..."
Northern Metropolitan,40,"{Alexkarbon, Rendina, YPA, Fletchers, Nick, Mi..."
Northern Victoria,11,"{McDonald, McGrath, Barry, YPA, Ray, HAR, othe..."
South-Eastern Metropolitan,25,"{Fletchers, Eview, C21, Hodges, Greg, Ray, Jel..."
Southern Metropolitan,38,"{Raine, Rendina, Fletchers, Nick, Eview, Phili..."
Western Metropolitan,34,"{Alexkarbon, Rendina, YPA, Village, Douglas, B..."
Western Victoria,6,"{YPA, Ray, HAR, other, hockingstuart, Raine}"


In [40]:
melb_df.groupby('Regionname')['Lattitude'].agg(
    ['mean', 'min', 'max']
).sort_values(by='max', ascending=False)

,mean,min,max
Regionname,,,
Northern Victoria,-37.599620,-37.68503,-37.40853
Western Metropolitan,-37.782382,-37.91859,-37.56005
Northern Metropolitan,-37.747258,-37.93940,-37.56857
Western Victoria,-37.695163,-37.71691,-37.67344
Eastern Metropolitan,-37.788820,-37.90383,-37.68508
Eastern Victoria,-37.959292,-38.16439,-37.75735
Southern Metropolitan,-37.866981,-37.99035,-37.78142
South-Eastern Metropolitan,-37.993727,-38.18255,-37.89210


In [41]:
melb_df.groupby('Regionname')['Lattitude'].agg('describe')

,count,mean,std,min,25%,50%,75%,max
Regionname,,,,,,,,
Eastern Metropolitan,1471.0,-37.788820,0.047890,-37.90383,-37.823185,-37.777000,-37.751800,-37.68508
Eastern Victoria,53.0,-37.959292,0.147067,-38.16439,-38.070540,-38.026210,-37.788830,-37.75735
Northern Metropolitan,3890.0,-37.747258,0.049639,-37.93940,-37.785100,-37.753380,-37.713625,-37.56857
Northern Victoria,41.0,-37.599620,0.084455,-37.68503,-37.672770,-37.638260,-37.507330,-37.40853
South-Eastern Metropolitan,450.0,-37.993727,0.073411,-38.18255,-38.028280,-37.981155,-37.931123,-37.89210
Southern Metropolitan,4695.0,-37.866981,0.043080,-37.99035,-37.901200,-37.858800,-37.835670,-37.78142
Western Metropolitan,2948.0,-37.782382,0.051251,-37.91859,-37.811885,-37.780400,-37.750275,-37.56005
Western Victoria,32.0,-37.695163,0.011579,-37.71691,-37.702827,-37.695075,-37.687370,-37.67344


In [44]:
#melb_df['Date'] = pd.to_datetime(melb_df['Date'])
display(melb_df['Date'])

0       2016-03-12
1       2016-04-02
2       2017-04-03
3       2017-04-03
4       2016-04-06
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]